In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
if CUDA_version == "10.0":
   torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
   torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
   torch_version_suffix = ""
else:
   torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install git+https://github.com/openai/CLIP.git
!pip install ftfy regex tqdm

CUDA version: 11.8
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2)
ERROR: No matching distribution found for torch=

In [3]:
from pathlib import Path
from torchvision.datasets.utils import download_url
import json
import torch
import clip
import os
from PIL import Image

# Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Directory of input images
Img_dir = "/content/drive/MyDrive/image_data/"

# Download ImageNet 1K class labels
if not Path("/content/data/imagenet_class_index.json").exists():
  download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

with open("/content/data/imagenet_class_index.json") as f:
  data = json.load(f)
  classes = [x["en"] for x in data]
  classes_japanese = [x["ja"] for x in data] #Japanese
  text = clip.tokenize([x["en"] for x in data]).to(device) #English

#three classes about hare
Angora_index = classes.index('Angora')
hare_index = classes.index('hare')
wood_rabbit_index = classes.index('wood_rabbit')
print(classes.index('tench'))

data_dir = Path(Img_dir)
data_files = data_dir.glob('*.png') #Images by .png name

for image_name in data_files:
  image = preprocess(Image.open(image_name)).unsqueeze(0).to(device)
  with torch.no_grad():

    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    #calculation of probabilities.
    logits_per_image, logits_per_text = model(image, text)
    probs, index_classes = logits_per_image.softmax(dim=-1).sort(dim=-1,descending=True)
    probs_rabbits = logits_per_image.softmax(dim=-1)

  #print image name
  print(image_name)

  #print up to the 10th probabile classes
  for k in range(10):
    prob = probs[0][k]
    prob_numpy = prob.cpu().numpy()
    indice = index_classes[0][k]
    indice_numpy = indice.cpu().numpy()
    class_name = classes[indice_numpy]
    class_name_japanese = classes_japanese[indice_numpy]
    print(k+1, indice_numpy,class_name, class_name_japanese, prob_numpy)

  print("")
  #print rabbit class (sum of the three hare class) and each hare class
  Angora_prob = probs_rabbits[0][Angora_index].cpu().numpy()
  hare_prob = probs_rabbits[0][hare_index].cpu().numpy()
  wood_rabbit_prob = probs_rabbits[0][wood_rabbit_index].cpu().numpy()
  print("Angora", Angora_index, classes_japanese[Angora_index], Angora_prob)
  print("hare", hare_index, classes_japanese[hare_index], hare_prob)
  print("wood_rabbit", wood_rabbit_index, classes_japanese[wood_rabbit_index], wood_rabbit_prob)
  print("all_rabbit", Angora_prob + hare_prob + wood_rabbit_prob)
  print("")


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 225MiB/s]


100%|██████████| 100891/100891 [00:00<00:00, 47339470.28it/s]

0


/content/drive/MyDrive/image_data/moon_binary_th90_resize_lat_0N.png
1 115 sea_slug ウミウシ 0.09344
2 305 dung_beetle フンコロガシ 0.0313
3 686 oil_filter オイルフィルター 0.02516
4 62 rock_python ロックパイソン 0.02185
5 712 Petri_dish ペトリ皿 0.02151
6 574 golf_ball ゴルフボール 0.01929
7 778 scale 規模 0.01929
8 980 volcano 火山 0.014114
9 65 sea_snake ウミヘビ 0.013054
10 347 bison バイソン 0.01285

Angora 332 アンゴラ 0.002867
hare 331 野ウサギ 0.000124
wood_rabbit 330 木のウサギ 0.00197
all_rabbit 0.00496

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_lat_0N.png
1 941 acorn_squash ドングリかぼちゃ 0.04614
2 332 Angora アンゴラ 0.01752
3 32 tailed_frog つかれたカエル 0.01697
4 267 standard_poodle スタンダードプードル 0.013855
5 66 horned_viper ツノクサリヘビ 0.01364
6 81 ptarmigan 雷鳥 0.01364
7 62 rock_python ロックパイソン 0.01322
8 192 cairn ケルン 0.01204
9 326 lycaenid シジミチョウ 0.01062
10 712 Petri_dish ペトリ皿 0.01046

Angora 332 アンゴラ 0.01752
hare 331 野ウサギ 0.000628
wood_rabbit 330 木のウサギ 0.002407
all_rabbit 0.02055

/content/drive/MyDrive/image_data/moon_binar